In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5

#games = pd.read_csv("IGDB_games.csv", sep=",")

In [4]:
ids_with_genres = pd.read_csv("../DataExtraction/ids_with_genres.csv", sep=",")
ids_with_genres 

,id,name
0,4,Fighting
1,5,Shooter
...,...,...
21,36,MOBA
22,2,Point-and-click


<h1> example with fake data </h1>

In [ ]:
# get Kelly's data queried as features
d = {'name': ['Skyrim', 'Destiny', 'Mario Kart', 'Forza'], 'adventure': [1, 0,0,0], 'racing': [0, 0,1,1], 'shooter': [0, 1,0,0]}


In [5]:
df = pd.DataFrame(data=d)
df
ids_with_genres

,id,name
0,4,Fighting
1,5,Shooter
...,...,...
21,36,MOBA
22,2,Point-and-click


In [6]:
# Features in our model. All quantitative, except Neighborhood.
features = ["adventure", "racing",
            "shooter"]

X_train_dict = df[features].to_dict(orient="records")
y_train = df["name"]

In [7]:
X_train = df[["adventure", "racing", "shooter"]]
y_train = df["name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [8]:
x_new = pd.Series(dtype=int)
x_new["adventure"] = 0
x_new["racing"] = 1
x_new["shooter"] = 0

x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

adventure   -0.500000
racing       0.866025
shooter     -0.500000
dtype: float64

In [584]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()
dists_sorted

2    0.000000
3    0.000000
0    2.645751
1    2.645751
dtype: float64

In [585]:
inds_nearest = dists_sorted.index[:9]
inds_nearest

Int64Index([2, 3, 0, 1], dtype='int64')

In [586]:
df.loc[inds_nearest]

,name,adventure,racing,shooter
2,Mario Kart,0,1,0
3,Forza,0,1,0
0,Skyrim,1,0,0
1,Destiny,0,0,1


<h1> example with 100 games </h1>
I can't work :(

In [587]:
# read the csv with all the games
games = pd.read_csv("test_dataset.csv", sep=",")

# split the string into a list of strings
games.genres = games.genres.str.split(', ').to_frame()

# fill in [0] for all the NaN values
games['genres'] = games['genres'].fillna({i: [0] for i in df.index})

games['genres'][97]

['0']

In [588]:
# convert the list of strings into a list of ints
for i, genre_list in enumerate(games.genres):
    for genre in genre_list:
        games.genres[i] = list(map(int, games.genres[i]))

<ipython-input-588-01529eb85361>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games.genres[i] = list(map(int, games.genres[i]))


In [589]:
# Getting distinct genre types for generating columns of genre type.
genre_columns = list(set([j for i in games['genres'].tolist() for j in i]))

In [590]:
# copy over name and id columns to new df
selected_columns = games[["id","name"]]
genre_df = selected_columns.copy()

In [591]:
# Iterating over every list to create and fill values into columns.
for j in genre_columns:
    genre_df[j] = 0
for i in range(games.shape[0]):
    for j in genre_columns:
        if(j in games['genres'].iloc[i]):
            genre_df.loc[i,j] = 1

In [592]:
genre_df

,id,name,0,2,4,5,8,9,10,12,...,15,16,24,25,30,31,32,33,34,35
0,73817,Spy Snatcher,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,112119,Woohoojin Bridges,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,86456,Battlezone 2 Remastered,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,17436,Momodora III,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [593]:
# Features in our model. All quantitative, except Neighborhood.
features = genre_columns

X_train_dict = genre_df[features].to_dict(orient="records")
y_train = genre_df["name"]

In [594]:
X_train = genre_df[genre_columns]
y_train = genre_df["name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [615]:
x_new = pd.Series(0, index=genre_columns)

x_new["9"] = 1
x_new["30"] = 1
x_new["31"] = 1

x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

0    -0.703526
2    -0.100504
        ...   
31         NaN
9          NaN
Length: 23, dtype: float64

In [616]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()
dists_sorted

0      2.110579
82     2.110579
        ...    
28    10.966808
3     11.016848
Length: 99, dtype: float64

In [617]:
inds_nearest = dists_sorted.index[:9]
inds_nearest

Int64Index([0, 82, 78, 33, 71, 35, 38, 39, 40], dtype='int64')

In [618]:
genre_df.loc[inds_nearest]

,id,name,0,2,4,5,8,9,10,12,...,15,16,24,25,30,31,32,33,34,35
0,73817,Spy Snatcher,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
82,111819,Cat Lady,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,76220,Dead by Daylight: The Last Breath Chapter,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,72929,Nova Usagi no Game de Ryuugaku!? DS,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
